In [2]:
# Install library yang dibutuhkan
# pip install nltk
# pip install Sastrawi
# pip install stanza
# pip install streamlit

import numpy as np
import pandas as pd
import re
import stanza
import nltk
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Set random seed
np.random.seed(42)

# Download data nltk
nltk.download('stopwords')
nltk.download('omw-1.4')

# Inisialisasi stopwords, stemmer, dan lemmatizer
stop_words = set(stopwords.words('indonesian'))
factory = StemmerFactory()
stemmer = factory.create_stemmer()
# Inisialisasi stanza untuk lemmatization
stanza.download('id')
nlp = stanza.Pipeline('id')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
2024-10-13 10:11:32 INFO: Downloaded file to C:\Users\ACER\stanza_resources\resources.json
2024-10-13 10:11:32 INFO: Downloading default packages for language: id (Indonesian) ...
2024-10-13 10:11:34 INFO: File exists: C:\Users\ACER\stanza_resources\id\default.zip
2024-10-13 10:11:38 INFO: Finished downloading models and saved to C:\Users\ACER\stanza_resources
2024-10-13 10:11:38 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-10-13 10:11:38 INFO: Downloaded file to C:\Users\ACER\stanza_resources\resources.json
2024-10-13 10:11:4

UI Dengan Streamlit

In [5]:
# Install library yang dibutuhkan
# Buat file app.py di dalam jupyter notebook
# eksekusi streamlit dengan aktivasi enviroment tempat streamlit diinstall (.venv\Scripts\python -m streamlit run app_vsm.py)

%%writefile app_vsm.py
import streamlit as st
import string
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean
import re
import nltk
import stanza
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Download NLTK data
nltk.download('stopwords')
nltk.download('omw-1.4')

# Initialize stopwords, stemmer, and lemmatizer
stop_words = set(stopwords.words('indonesian'))
factory = StemmerFactory()
stemmer = factory.create_stemmer()
stanza.download('id')
nlp = stanza.Pipeline('id')

# Preprocessing functions
def case_folding(text):
    text = text.lower()
    return re.sub(r'[^\w\s]', '', text)

def tokenize(text):
    return text.split()

def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

def stemming(tokens):
    return [stemmer.stem(word) for word in tokens]

def lemmatization(tokens):
    text = ' '.join(tokens)
    doc = nlp(text)
    return [word.lemma for sent in doc.sentences for word in sent.words]

def preprocess_with_stemming(text):
    text = case_folding(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = stemming(tokens)
    return ' '.join(tokens)

def preprocess_with_lemmatization(text):
    text = case_folding(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = lemmatization(tokens)
    return ' '.join(tokens)

# TF-IDF functions
def tfidf_with_stemming(documents):
    preprocessed_documents = [preprocess_with_stemming(doc) for doc in documents]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(preprocessed_documents)
    return tfidf_matrix, vectorizer

def tfidf_with_lemmatization(documents):
    preprocessed_documents = [preprocess_with_lemmatization(doc) for doc in documents]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(preprocessed_documents)
    return tfidf_matrix, vectorizer

# TF and IDF computation functions
def compute_tf(documents, tf_type):
    tf_matrix = []
    for doc in documents:
        tokens = preprocess_with_lemmatization(doc)
        token_counts = pd.Series(tokens.split()).value_counts()

        if tf_type == 'binary':
            tf_vector = (token_counts > 0).astype(int)
        elif tf_type == 'raw_frequency':
            tf_vector = token_counts
        elif tf_type == 'log_normalization':
            tf_vector = 1 + np.log(token_counts)
        elif tf_type == 'double_normalization_0.5':
            max_count = token_counts.max()
            tf_vector = 0.5 + 0.5 * (token_counts / max_count)
        elif tf_type == 'double_normalization_K':
            K = 0.5
            max_count = token_counts.max()
            tf_vector = K + (1 - K) * (token_counts / max_count)

        tf_matrix.append(tf_vector)

    return pd.DataFrame(tf_matrix).fillna(0).T

def compute_idf(documents, idf_type):
    N = len(documents)
    token_df = pd.Series(' '.join(preprocess_with_lemmatization(doc) for doc in documents).split()).value_counts()

    if idf_type == 'unary':
        idf_vector = pd.Series(1, index=token_df.index)
    elif idf_type == 'inverse_frequency':
        idf_vector = np.log(N / token_df)
    elif idf_type == 'inv_frequency_smooth':
        idf_vector = np.log(1 + (N / token_df))
    elif idf_type == 'inv_frequency_max':
        max_df = token_df.max()
        idf_vector = np.log(1 + (max_df / token_df))
    elif idf_type == 'probabilistic_inv_frequency':
        idf_vector = np.log((N - token_df) / token_df)

    return idf_vector

def compute_tf_idf(documents, tf_type, idf_type):
    tf_matrix = compute_tf(documents, tf_type)
    idf_vector = compute_idf(documents, idf_type)
    tf_idf_matrix = tf_matrix.multiply(idf_vector, axis=0)
    return tf_idf_matrix

# Search function
def search_documents(documents, query, tf_type, idf_type, similarity_metric):
    tf_idf_matrix = compute_tf_idf(documents, tf_type, idf_type)
    
    preprocessed_query = preprocess_with_lemmatization(query)
    query_tf = pd.Series(preprocessed_query.split()).value_counts()
    query_tf_vector = pd.Series(query_tf).reindex(tf_idf_matrix.index, fill_value=0)

    if similarity_metric == 'cosine':
        similarities = cosine_similarity(query_tf_vector.values.reshape(1, -1), tf_idf_matrix.T).flatten()
    elif similarity_metric == 'euclidean':
        similarities = np.array([euclidean(query_tf_vector, doc_vector) for doc_vector in tf_idf_matrix.T.to_numpy()])
        similarities = 1 / (1 + similarities)  # Convert to similarity score (higher is better)
    
    ranked_indices = np.argsort(similarities)[::-1]
    return ranked_indices, similarities

# Streamlit app
def main():
    st.title("Vector Space Model Document Search")
    
    # Input for documents
    documents = st.text_area("Enter your documents (one per line):", height=200)
    documents = documents.split('\n')
    
    # Input for query
    query = st.text_input("Enter your search query:")
    
    # Select TF type
    tf_type = st.selectbox(
        "Select Term Frequency (TF) type:",
        ['binary', 'raw_frequency', 'log_normalization', 'double_normalization_0.5', 'double_normalization_K']
    )
    
    # Select IDF type
    idf_type = st.selectbox(
        "Select Inverse Document Frequency (IDF) type:",
        ['unary', 'inverse_frequency', 'inv_frequency_smooth', 'inv_frequency_max', 'probabilistic_inv_frequency']
    )
    
    # Select similarity metric
    similarity_metric = st.radio("Select similarity metric:", ["Cosine Similarity", "Euclidean Distance"])
    
    if st.button("Search"):
        if documents and query:
            ranked_indices, similarities = search_documents(
                documents, 
                query, 
                tf_type, 
                idf_type, 
                'cosine' if similarity_metric == "Cosine Similarity" else 'euclidean'
            )
            
            st.subheader("Search Results:")
            for i, idx in enumerate(ranked_indices[:10]):  # Display top 10 results
                score = similarities[idx]
                st.write(f"{i+1}. ({similarity_metric}: {score:.4f}) {documents[idx]}")
        else:
            st.warning("Please enter both documents and a search query.")
    
    # Preprocessing demonstration
    if st.checkbox("Show Preprocessing Results"):
        st.subheader("Preprocessing Results:")
        for doc in documents:
            original_text = case_folding(doc)
            tokens = tokenize(original_text)
            tokens_no_stopwords = remove_stopwords(tokens)
            stemmed_result = stemming(tokens_no_stopwords)
            lemmatized_result = lemmatization(tokens_no_stopwords)
            
            st.write(f"Original: {original_text}")
            st.write(f"Stemming: {' '.join(stemmed_result)}")
            st.write(f"Lemmatization: {' '.join(lemmatized_result)}")
            st.write("\n")

if __name__ == "__main__":
    main()

Overwriting app_vsm.py


LOAD DATA

In [2]:
documents = [
    "Perpustakaan merupakan tempat yang penting untuk belajar dan mendapatkan informasi.",
    "Membaca buku di perpustakaan memberikan pengalaman belajar yang berbeda.",
    "Kalangan mahasiswa sering mengunjungi perpustakaan untuk belajar.",
    "Koleksi buku di perpustakaan sangat beragam dan mencakup berbagai disiplin ilmu.",
    "Belajar mandiri dapat dilakukan dengan memanfaatkan sumber daya perpustakaan.",
    "Perpustakaan menyediakan berbagai fasilitas untuk mendukung proses belajar.",
    "Kalangan pelajar di sekolah juga sering menggunakan perpustakaan sebagai tempat belajar.",
    "Perpustakaan digital memudahkan akses informasi bagi semua kalangan.",
    "Belajar kelompok di perpustakaan dapat meningkatkan pemahaman materi.",
    "Perpustakaan umum juga mengadakan berbagai acara untuk meningkatkan minat baca kalangan masyarakat.",
    "Pustakawan berperan penting dalam membantu pengunjung mencari informasi untuk belajar.",
    "Buku referensi di perpustakaan sangat membantu dalam menyusun tugas dan penelitian.",
    "Kalangan akademisi sering memanfaatkan perpustakaan untuk penelitian dan pengembangan ilmu.",
    "Perpustakaan memiliki sistem pengelolaan yang baik untuk menjaga koleksi buku.",
    "Belajar melalui media elektronik di perpustakaan semakin populer di kalangan remaja.",
    "Perpustakaan memainkan peran penting dalam mendukung program literasi di masyarakat.",
    "Kalangan profesional juga dapat mengakses jurnal dan artikel ilmiah di perpustakaan.",
    "Membaca majalah dan koran di perpustakaan dapat memberikan wawasan baru bagi pengunjung.",
    "Perpustakaan sekolah sering kali menjadi tempat yang nyaman untuk belajar.",
    "Kalangan peneliti dapat menggunakan perpustakaan untuk mendapatkan data dan referensi yang akurat."
]


PREPROCESSING TEXT

In [3]:
# Step 1: Case folding and cleaning (used in all scenarios)
def case_folding(text):
    text = text.lower()
    return re.sub(r'[^\w\s]', '', text)

# Step 2: Tokenization
def tokenize(text):
    return text.split()

# Step 3: Stopwords removal
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

# Step 4: Stemming
def stemming(tokens):
    return [stemmer.stem(word) for word in tokens]

# Step 5: Lemmatization using stanza
def lemmatization(tokens):
    text = ' '.join(tokens)  # Gabungkan token menjadi teks untuk diproses
    doc = nlp(text)
    return [word.lemma for sent in doc.sentences for word in sent.words]

In [4]:
# Preprocessing with stemming
def preprocess_with_stemming(text):
    text = case_folding(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = stemming(tokens)
    return ' '.join(tokens)

In [5]:
# Preprocessing with lemmatization
def preprocess_with_lemmatization(text):
    text = case_folding(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = lemmatization(tokens)
    return ' '.join(tokens)

In [6]:
# Menerapkan preprocessing pada setiap dokumen
print("Hasil Preprocessing dengan Stemming dan Lemmatization:")
for doc in documents:
    # Proses case folding
    original_text = case_folding(doc)

    # Tokenisasi
    tokens = tokenize(original_text)

    # Stopwords removal
    tokens_no_stopwords = remove_stopwords(tokens)

    # Proses stemming
    stemmed_result = stemming(tokens_no_stopwords)

    # Proses lemmatization
    lemmatized_result = lemmatization(tokens_no_stopwords)

    # Menampilkan hasil
    print(f"Original: {original_text}")
    print(f"Stemming: {' '.join(stemmed_result)}")
    print(f"Lemmatization: {' '.join(lemmatized_result)}")
    print("\n")

Hasil Preprocessing dengan Stemming dan Lemmatization:
Original: perpustakaan merupakan tempat yang penting untuk belajar dan mendapatkan informasi
Stemming: pustaka ajar informasi
Lemmatization: perpustakaan belajar informasi


Original: membaca buku di perpustakaan memberikan pengalaman belajar yang berbeda
Stemming: baca buku pustaka alam ajar beda
Lemmatization: baca buku perpustakaan pengalaman belajar beda


Original: kalangan mahasiswa sering mengunjungi perpustakaan untuk belajar
Stemming: kalang mahasiswa unjung pustaka ajar
Lemmatization: kalangan mahasiswa kunjung perpustakaan belajar


Original: koleksi buku di perpustakaan sangat beragam dan mencakup berbagai disiplin ilmu
Stemming: koleksi buku pustaka agam cakup disiplin ilmu
Lemmatization: koleksi buku perpustakaan ragam cakup disiplin ilmu


Original: belajar mandiri dapat dilakukan dengan memanfaatkan sumber daya perpustakaan
Stemming: ajar mandiri manfaat sumber daya pustaka
Lemmatization: belajar mandiri manfaat sum

PEMBOBOTAN KATA

In [7]:
# Fungsi untuk membuat TF-IDF dengan stemming
def tfidf_with_stemming(documents):
    # Preprocessing dengan stemming
    preprocessed_documents = [preprocess_with_stemming(doc) for doc in documents]
    # Buat TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(preprocessed_documents)
    return tfidf_matrix, vectorizer

In [8]:
# Fungsi untuk membuat TF-IDF dengan lemmatization

def tfidf_with_lemmatization(documents):
    # Preprocessing dengan lemmatization
    preprocessed_documents = [preprocess_with_lemmatization(doc) for doc in documents]
    # Buat TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(preprocessed_documents)
    return tfidf_matrix, vectorizer

QUERY UNTUK PENCARIAN DOKUMEN

In [9]:
query = "Mahasiswa dari berbagai kalangan berkunjung ke perpustakaan untuk belajar"

# SKENARIO 1 : STEMMING VS LEMMATIZATION

In [ ]:
# Scenario 1a: Stemming
print("\nScenario 1a: Using Stemming")
tfidf_stemmed_matrix, tfidf_stemmed_vectorizer = tfidf_with_stemming(documents)

# Preprocess query with stemming
preprocessed_query_stemmed = preprocess_with_stemming(query)
query_vector_stemmed = tfidf_stemmed_vectorizer.transform([preprocessed_query_stemmed])

# Calculate cosine similarities
cosine_similarities_stemmed = cosine_similarity(query_vector_stemmed, tfidf_stemmed_matrix).flatten()

# Rank documents based on similarities
ranked_indices_stemmed = np.argsort(cosine_similarities_stemmed)[::-1]

# Tampilkan hasil
print("Pencarian dengan TF-IDF (Stemming):")
print("Query:", query)
print("\nTop Documents:")

# Perbaikan: definisikan i sebagai counter
for i, idx in enumerate(ranked_indices_stemmed):
    print(f"{i + 1}. Doc {idx + 1} \t (Cosine Similarity: {cosine_similarities_stemmed[idx]:.4f}) \t: {documents[idx]}")



Scenario 1a: Using Stemming
Pencarian dengan TF-IDF (Stemming):
Query: Mahasiswa dari berbagai kalangan berkunjung ke perpustakaan untuk belajar

Top Documents:
1. Doc 3 	 (Cosine Similarity: 0.8504) 	: Kalangan mahasiswa sering mengunjungi perpustakaan untuk belajar.
2. Doc 7 	 (Cosine Similarity: 0.4717) 	: Kalangan pelajar di sekolah juga sering menggunakan perpustakaan sebagai tempat belajar.
3. Doc 1 	 (Cosine Similarity: 0.2690) 	: Perpustakaan merupakan tempat yang penting untuk belajar dan mendapatkan informasi.
4. Doc 15 	 (Cosine Similarity: 0.2328) 	: Belajar melalui media elektronik di perpustakaan semakin populer di kalangan remaja.
5. Doc 8 	 (Cosine Similarity: 0.1607) 	: Perpustakaan digital memudahkan akses informasi bagi semua kalangan.
6. Doc 20 	 (Cosine Similarity: 0.1607) 	: Kalangan peneliti dapat menggunakan perpustakaan untuk mendapatkan data dan referensi yang akurat.
7. Doc 19 	 (Cosine Similarity: 0.1501) 	: Perpustakaan sekolah sering kali menjadi tempat y

In [11]:
# Scenario 1b: Lemmatization
print("\nScenario 1b: Using Lemmatization")
tfidf_lemmatized_matrix, tfidf_lemmatized_vectorizer = tfidf_with_lemmatization(documents)

# Preprocess query with lemmatization
preprocessed_query_lemmatized = preprocess_with_lemmatization(query)
query_vector_lemmatized = tfidf_lemmatized_vectorizer.transform([preprocessed_query_lemmatized])

# Calculate cosine similarities
cosine_similarities_lemmatized = cosine_similarity(query_vector_lemmatized, tfidf_lemmatized_matrix).flatten()

# Rank documents based on similarities
ranked_indices_lemmatized = np.argsort(cosine_similarities_lemmatized)[::-1]

# Tampilkan hasil dengan penomoran
print("Pencarian dengan TF-IDF (Lemmatization):")
print("Query:", query)
print("\nTop Documents:")
for i, idx in enumerate(ranked_indices_lemmatized):
    print(f"{i + 1}. Doc {idx + 1} \t (Cosine Similarity: {cosine_similarities_lemmatized[idx]:.4f}) \t: {documents[idx]}")



Scenario 1b: Using Lemmatization
Pencarian dengan TF-IDF (Lemmatization):
Query: Mahasiswa dari berbagai kalangan berkunjung ke perpustakaan untuk belajar

Top Documents:
1. Doc 3 	 (Cosine Similarity: 1.0000) 	: Kalangan mahasiswa sering mengunjungi perpustakaan untuk belajar.
2. Doc 7 	 (Cosine Similarity: 0.2745) 	: Kalangan pelajar di sekolah juga sering menggunakan perpustakaan sebagai tempat belajar.
3. Doc 11 	 (Cosine Similarity: 0.2571) 	: Pustakawan berperan penting dalam membantu pengunjung mencari informasi untuk belajar.
4. Doc 18 	 (Cosine Similarity: 0.2311) 	: Membaca majalah dan koran di perpustakaan dapat memberikan wawasan baru bagi pengunjung.
5. Doc 1 	 (Cosine Similarity: 0.2287) 	: Perpustakaan merupakan tempat yang penting untuk belajar dan mendapatkan informasi.
6. Doc 15 	 (Cosine Similarity: 0.1980) 	: Belajar melalui media elektronik di perpustakaan semakin populer di kalangan remaja.
7. Doc 20 	 (Cosine Similarity: 0.1367) 	: Kalangan peneliti dapat menggu

# SKENARIO 2 : COSINE SIMILARITY VS EUCLIDEAN DISTANCE

In [12]:
# Scenario 2a: Using Euclidean Distance
print("\nScenario 2a: Using Euclidean Distance")
tfidf_lemmatized_matrix, tfidf_lemmatized_vectorizer = tfidf_with_lemmatization(documents)

# Preprocess query with lemmatization
preprocessed_query_lemmatized = preprocess_with_lemmatization(query)
query_vector_lemmatized = tfidf_lemmatized_vectorizer.transform([preprocessed_query_lemmatized])

# Convert the sparse matrices to dense arrays
tfidf_lemmatized_matrix_dense = tfidf_lemmatized_matrix.toarray()
query_vector_lemmatized_dense = query_vector_lemmatized.toarray()

# Calculate Euclidean distances
euclidean_distances_lemmatized = np.linalg.norm(tfidf_lemmatized_matrix_dense - query_vector_lemmatized_dense, axis=1)

# Rank documents based on Euclidean Distance (nilai lebih kecil lebih baik)
ranked_indices_lemmatized = np.argsort(euclidean_distances_lemmatized)

# Tampilkan hasil dengan penomoran
print("Pencarian dengan TF-IDF (Euclidean Distance):")
print("Query:", query)
print("\nTop Documents:")
for i, idx in enumerate(ranked_indices_lemmatized):
    print(f"{i + 1}. Doc {idx + 1} \t (Euclidean Distance: {euclidean_distances_lemmatized[idx]:.4f}) \t: {documents[idx]}")



Scenario 2a: Using Euclidean Distance
Pencarian dengan TF-IDF (Euclidean Distance):
Query: Mahasiswa dari berbagai kalangan berkunjung ke perpustakaan untuk belajar

Top Documents:
1. Doc 3 	 (Euclidean Distance: 0.0000) 	: Kalangan mahasiswa sering mengunjungi perpustakaan untuk belajar.
2. Doc 7 	 (Euclidean Distance: 1.2046) 	: Kalangan pelajar di sekolah juga sering menggunakan perpustakaan sebagai tempat belajar.
3. Doc 11 	 (Euclidean Distance: 1.2189) 	: Pustakawan berperan penting dalam membantu pengunjung mencari informasi untuk belajar.
4. Doc 18 	 (Euclidean Distance: 1.2401) 	: Membaca majalah dan koran di perpustakaan dapat memberikan wawasan baru bagi pengunjung.
5. Doc 1 	 (Euclidean Distance: 1.2420) 	: Perpustakaan merupakan tempat yang penting untuk belajar dan mendapatkan informasi.
6. Doc 15 	 (Euclidean Distance: 1.2665) 	: Belajar melalui media elektronik di perpustakaan semakin populer di kalangan remaja.
7. Doc 20 	 (Euclidean Distance: 1.3140) 	: Kalangan pene

In [13]:
# Scenario 2b: Using Cosine Similarity
print("\nScenario 1b: Using Cosine Similarity")
tfidf_lemmatized_matrix, tfidf_lemmatized_vectorizer = tfidf_with_lemmatization(documents)

# Preprocess query with lemmatization
preprocessed_query_lemmatized = preprocess_with_lemmatization(query)
query_vector_lemmatized = tfidf_lemmatized_vectorizer.transform([preprocessed_query_lemmatized])

# Calculate Cosine Similarities
cosine_similarities_lemmatized = cosine_similarity(query_vector_lemmatized, tfidf_lemmatized_matrix).flatten()

# Rank documents based on Cosine Similarity (nilai lebih besar lebih baik)
ranked_indices_lemmatized = np.argsort(cosine_similarities_lemmatized)[::-1]

# Tampilkan hasil dengan penomoran
print("Pencarian dengan TF-IDF (Cosine Similarity):")
print("Query:", query)
print("\nTop Documents:")
for i, idx in enumerate(ranked_indices_lemmatized):
    print(f"{i + 1}. Doc {idx + 1} \t (Cosine Similarity: {cosine_similarities_lemmatized[idx]:.4f}) \t: {documents[idx]}")



Scenario 1b: Using Cosine Similarity
Pencarian dengan TF-IDF (Cosine Similarity):
Query: Mahasiswa dari berbagai kalangan berkunjung ke perpustakaan untuk belajar

Top Documents:
1. Doc 3 	 (Cosine Similarity: 1.0000) 	: Kalangan mahasiswa sering mengunjungi perpustakaan untuk belajar.
2. Doc 7 	 (Cosine Similarity: 0.2745) 	: Kalangan pelajar di sekolah juga sering menggunakan perpustakaan sebagai tempat belajar.
3. Doc 11 	 (Cosine Similarity: 0.2571) 	: Pustakawan berperan penting dalam membantu pengunjung mencari informasi untuk belajar.
4. Doc 18 	 (Cosine Similarity: 0.2311) 	: Membaca majalah dan koran di perpustakaan dapat memberikan wawasan baru bagi pengunjung.
5. Doc 1 	 (Cosine Similarity: 0.2287) 	: Perpustakaan merupakan tempat yang penting untuk belajar dan mendapatkan informasi.
6. Doc 15 	 (Cosine Similarity: 0.1980) 	: Belajar melalui media elektronik di perpustakaan semakin populer di kalangan remaja.
7. Doc 20 	 (Cosine Similarity: 0.1367) 	: Kalangan peneliti dapa

# SKENARIO 3 : VARIASI TF

In [14]:
# Fungsi untuk menghitung berbagai jenis Term Frequency (TF)
def compute_tf(documents, tf_type):
    tf_matrix = []
    for doc in documents:
        # Preprocess dokumen menggunakan lemmatization
        tokens = preprocess_with_lemmatization(doc)
        token_counts = pd.Series(tokens.split()).value_counts()

        # Hitung jenis TF sesuai dengan tipe yang dipilih
        if tf_type == 'binary':
            tf_vector = (token_counts > 0).astype(int)
        elif tf_type == 'raw_frequency':
            tf_vector = token_counts
        elif tf_type == 'log_normalization':
            tf_vector = 1 + np.log(token_counts)
        elif tf_type == 'double_normalization_0.5':
            max_count = token_counts.max()
            tf_vector = 0.5 + 0.5 * (token_counts / max_count)
        elif tf_type == 'double_normalization_K':
            K = 0.5  # Anda dapat mengubah nilai K sesuai kebutuhan
            max_count = token_counts.max()
            tf_vector = K + (1 - K) * (token_counts / max_count)

        # Tambahkan ke matriks TF
        tf_matrix.append(tf_vector)

    # Kembalikan DataFrame dengan nilai 0 untuk nilai yang hilang (fillna(0))
    return pd.DataFrame(tf_matrix).fillna(0).T

# Fungsi untuk menjalankan TF dan pencarian dokumen
def run_tf_ranking(documents, query, tf_types):
    for tf_type in tf_types:
        print(f"\nScenario: Using TF = {tf_type}")

        # Hitung TF untuk dokumen
        tf_matrix = compute_tf(documents, tf_type)

        # Preprocess query menggunakan lemmatization
        preprocessed_query_tf = preprocess_with_lemmatization(query)
        query_tf = pd.Series(preprocessed_query_tf.split()).value_counts()

        # Buat query vector yang sejajar dengan matriks TF dokumen
        query_tf_vector = pd.Series(query_tf).reindex(tf_matrix.index, fill_value=0).values.reshape(1, -1)

        # Hitung cosine similarity
        cosine_similarities_tf = cosine_similarity(query_tf_vector, tf_matrix.T).flatten()

        # Ranking dokumen berdasarkan cosine similarity
        ranked_indices_tf = np.argsort(cosine_similarities_tf)[::-1]

        # Tampilkan hasil ranking dokumen dengan penomoran
        print(f"Pencarian dengan TF = {tf_type}:")
        print("Query:", query)
        print("\nTop Documents:")
        for i, idx in enumerate(ranked_indices_tf):
            print(f"{i + 1}. Doc {idx + 1} \t (Cosine Similarity: {cosine_similarities_tf[idx]:.4f}) \t: {documents[idx]}")

# Kombinasi berbagai jenis TF
tf_types = ['binary', 'raw_frequency', 'log_normalization', 'double_normalization_0.5', 'double_normalization_K']

# Jalankan ranking dengan semua variasi TF
run_tf_ranking(documents, query, tf_types)



Scenario: Using TF = binary
Pencarian dengan TF = binary:
Query: Mahasiswa dari berbagai kalangan berkunjung ke perpustakaan untuk belajar

Top Documents:
1. Doc 3 	 (Cosine Similarity: 1.0000) 	: Kalangan mahasiswa sering mengunjungi perpustakaan untuk belajar.
2. Doc 7 	 (Cosine Similarity: 0.6000) 	: Kalangan pelajar di sekolah juga sering menggunakan perpustakaan sebagai tempat belajar.
3. Doc 1 	 (Cosine Similarity: 0.5164) 	: Perpustakaan merupakan tempat yang penting untuk belajar dan mendapatkan informasi.
4. Doc 15 	 (Cosine Similarity: 0.5071) 	: Belajar melalui media elektronik di perpustakaan semakin populer di kalangan remaja.
5. Doc 19 	 (Cosine Similarity: 0.4000) 	: Perpustakaan sekolah sering kali menjadi tempat yang nyaman untuk belajar.
6. Doc 2 	 (Cosine Similarity: 0.3651) 	: Membaca buku di perpustakaan memberikan pengalaman belajar yang berbeda.
7. Doc 20 	 (Cosine Similarity: 0.3651) 	: Kalangan peneliti dapat menggunakan perpustakaan untuk mendapatkan data dan

# SKENARIO 4 : VARIASI TF-IDF

In [15]:
# Fungsi untuk menghitung berbagai jenis TF
def compute_tf(documents, tf_type):
    tf_matrix = []
    for doc in documents:
        # Preprocess dokumen menggunakan lemmatization
        tokens = preprocess_with_lemmatization(doc)
        token_counts = pd.Series(tokens.split()).value_counts()

        # Hitung jenis TF sesuai dengan tipe yang dipilih
        if tf_type == 'binary':
            tf_vector = (token_counts > 0).astype(int)
        elif tf_type == 'raw_frequency':
            tf_vector = token_counts
        elif tf_type == 'log_normalization':
            tf_vector = 1 + np.log(token_counts)
        elif tf_type == 'double_normalization_0.5':
            max_count = token_counts.max()
            tf_vector = 0.5 + 0.5 * (token_counts / max_count)
        elif tf_type == 'double_normalization_K':
            K = 0.5  # Anda dapat mengubah nilai K sesuai kebutuhan
            max_count = token_counts.max()
            tf_vector = K + (1 - K) * (token_counts / max_count)

        # Tambahkan ke matriks TF
        tf_matrix.append(tf_vector)

    # Kembalikan DataFrame dengan nilai 0 untuk nilai yang hilang (fillna(0))
    return pd.DataFrame(tf_matrix).fillna(0).T

# Fungsi untuk menghitung berbagai jenis IDF
def compute_idf(documents, idf_type):
    N = len(documents)
    token_df = pd.Series(' '.join(preprocess_with_lemmatization(doc) for doc in documents).split()).value_counts()

    # Hitung jenis IDF sesuai dengan tipe yang dipilih
    if idf_type == 'unary':
        idf_vector = pd.Series(1, index=token_df.index)
    elif idf_type == 'inverse_frequency':
        idf_vector = np.log(N / token_df)
    elif idf_type == 'inv_frequency_smooth':
        idf_vector = np.log(1 + (N / token_df))
    elif idf_type == 'inv_frequency_max':
        max_df = token_df.max()
        idf_vector = np.log(1 + (max_df / token_df))
    elif idf_type == 'probabilistic_inv_frequency':
        idf_vector = np.log((N - token_df) / token_df)

    return idf_vector

# Fungsi untuk menghitung TF-IDF
def compute_tf_idf(documents, tf_type, idf_type):
    tf_matrix = compute_tf(documents, tf_type)
    idf_vector = compute_idf(documents, idf_type)

    # Perkalian elemen-wise antara TF dan IDF
    tf_idf_matrix = tf_matrix.multiply(idf_vector, axis=0)

    return tf_idf_matrix

# Kombinasi berbagai jenis TF dan IDF
tf_types = ['binary', 'raw_frequency', 'log_normalization', 'double_normalization_0.5', 'double_normalization_K']
idf_types = ['unary', 'inverse_frequency', 'inv_frequency_smooth', 'inv_frequency_max', 'probabilistic_inv_frequency']

# Fungsi utama untuk menjalankan kombinasi TF-IDF dan perankingan dokumen
def run_tf_idf_ranking(documents, query, tf_types, idf_types):
    for tf_type in tf_types:
        for idf_type in idf_types:
            print(f"\nScenario: Using TF = {tf_type} and IDF = {idf_type}")

            # Hitung TF-IDF untuk dokumen
            tf_idf_matrix = compute_tf_idf(documents, tf_type, idf_type)

            # Preprocess query menggunakan lemmatization
            preprocessed_query = preprocess_with_lemmatization(query)
            query_tf = pd.Series(preprocessed_query.split()).value_counts()

            # Hitung TF-IDF untuk query dan cosine similarity
            query_tf_vector = pd.Series(query_tf).reindex(tf_idf_matrix.index, fill_value=0).values.reshape(1, -1)
            cosine_similarities = cosine_similarity(query_tf_vector, tf_idf_matrix.T).flatten()

            # Ranking dokumen berdasarkan cosine similarity
            ranked_indices = np.argsort(cosine_similarities)[::-1]

            # Tampilkan hasil ranking dokumen dengan format sesuai keinginan Anda
            print(f"Pencarian dengan TF = {tf_type} dan IDF = {idf_type}:")
            print("Query:", query)
            print("\nTop Documents:")
            for i, idx in enumerate(ranked_indices):
                print(f"{i + 1}. Doc {idx + 1} \t (Cosine Similarity: {cosine_similarities[idx]:.4f}) \t: {documents[idx]}")

# Contoh penggunaan dengan query
run_tf_idf_ranking(documents, query, tf_types, idf_types)



Scenario: Using TF = binary and IDF = unary
Pencarian dengan TF = binary dan IDF = unary:
Query: Mahasiswa dari berbagai kalangan berkunjung ke perpustakaan untuk belajar

Top Documents:
1. Doc 3 	 (Cosine Similarity: 1.0000) 	: Kalangan mahasiswa sering mengunjungi perpustakaan untuk belajar.
2. Doc 7 	 (Cosine Similarity: 0.6000) 	: Kalangan pelajar di sekolah juga sering menggunakan perpustakaan sebagai tempat belajar.
3. Doc 1 	 (Cosine Similarity: 0.5164) 	: Perpustakaan merupakan tempat yang penting untuk belajar dan mendapatkan informasi.
4. Doc 15 	 (Cosine Similarity: 0.5071) 	: Belajar melalui media elektronik di perpustakaan semakin populer di kalangan remaja.
5. Doc 19 	 (Cosine Similarity: 0.4000) 	: Perpustakaan sekolah sering kali menjadi tempat yang nyaman untuk belajar.
6. Doc 2 	 (Cosine Similarity: 0.3651) 	: Membaca buku di perpustakaan memberikan pengalaman belajar yang berbeda.
7. Doc 20 	 (Cosine Similarity: 0.3651) 	: Kalangan peneliti dapat menggunakan perpust